<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb tenacity

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import random
from pathlib import Path
import openai
import tiktoken
from pprint import pprint
from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [5]:
# start logging to W&B
autolog({"project":"llmapps", "entity":"cmagganas", "job_type": "generation"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cmagganas. Use `wandb login --relogin` to force relogin


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [7]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [8]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

How can I save a trained model in W&B and access it later for inference or further training?

"How can I customize my project dashboard on Weights & Biases to better visualize my experiment results?"

"How can I track multiple experiments and compare their results easily on the W&B platform?"

"How can I track and compare model performance between different experiments in W&B?"

"How can I use Weights and Biases to track and compare the performance of multiple machine learning models over    
time?"

# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [9]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")  
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "can I use the column subsample parameter config file for xgboost parameters through sweeps?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [10]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: weights &     
biases have released a new product called wandb-geofinder to improve logging geographical data to wandb. Using     
wandb-geofinder, what is the capital of Ireland I was trying to save model checkpoints using wandb.save(path) where
my path is ~/.save_folder (notice the dot), I am getting an error. I am running on windows. I Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [11]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

Sure, here's a possible support question from a W&B user:                                                          

"I am trying to run a hyperparameter sweep with W&B sweeps feature, but I keep getting an error when trying to run 
my script on a remote machine. I have set up my API key and configured my sweep YAML file correctly, but I can't   
seem to figure out why my script won't run properly. Can you help me troubleshoot this issue?"

Sure, here's a support question a W&B user might have:                                                             

I'm trying to use wandb.log() to log my model's training metrics, but I keep getting an error that says "wandb:    
ERROR Not authenticated". How do I authenticate my account in order to use wandb.log()?

Sure thing! Here's a possible support question from a W&B user:                                                    

"I'm using W&B to log my experiments, and I noticed that some of the plots and graphs are not showing up on my     
dashboard, even though the runs seem to be getting logged correctly. What could be causing this issue?"

Here's a support question from a W&B user: I'm using W&B for my deep learning project and I want to track the      
performance of my model with different hyperparameters. How can I easily compare the results?

How do I integrate W&B with my machine learning framework?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [12]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [13]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[2093, 2940, 2529, 1644, 803, 2596, 1206, 365, 537, 4179, 956]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [14]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [15]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [16]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: the Team dropdown menu, and select a role type 
from the Organizational Role dropdown menu.                                                                        

🌆 ]8;id=355995;@site/static/images/app_ui/ezgif-3-b665ff2fa9.gif\ezgif-3-b665ff2fa9.gif]8;;\                                                                                                                    

 5 Select the Add button.                                                                                          

:::info                                                                                                            

 • If you have an Enterprise account, please contact your Account Executive to invite new members to your team. :::


                                               Create a Team Profile                                               

You can customize your team's profile page to show an introduction and showcase reports and projects that are      
visible to the public or team members. Present reports, projects, and external links.                              

 • Highlight your best research to visitors by showcasing your best public reports                                 
 • Showcase the most active projects to make it easier for teammates to find them                                  
 • Find collaborators by adding external links to your company or research lab's website and any papers you've     
   published                                                                                                       




                                                Remove team members                                                

Team admins can open the team settings page and click the delete button next to the departing member's name. Any   
runs that they logged to the team will remain after a user is removed.                                             


                                            Team Roles and Permissions                                             

Select one a team role when you invite colleagues to join a team. There are four team role options:                

 • Admin: Team admins can add and remove other admins or team members. They have permissions to modify all projects
   and full deletion permissions. This includes, but is not limited to, deleting runs, projects, artifacts, and    
   sweeps.                                                                                                         
 • Member: A regular member of the team. A team member is invited by email by the team admin. A team member cannot 
   invite other members. Team members can only delete runs and sweep runs created by that member. Suppose you have 
   two members A and B. Member B moves a Run from team B's project to a different project owned by Member A. Member
   A can not delete the Run Member B moved to Member A's project. Only the member that creates the Run, or the team
   admin, can delete the run.                                                                                      
 • Service: A service worker, an API key useful for using W&B with your run automation tools. If you use the API   
   key from a service account for your team, make sure to set the environment variable WANDB_USERNAME to attribute 
   runs to the correct user.                                                                                       
 • View-Only (Enterprise-only feature): View-Only members can view assets within the team such as runs, reports,   
   and workspaces. They can Let's start!

Let's generate 3 possible questions:

In [17]:
generate_and_print(system_prompt, generation_prompt, n=3)

What are the different roles available when inviting colleagues to join a team and what are their corresponding    
permissions in W&B?

What are the different team roles in W&B and what are their permissions?

What are the different team roles and permissions in W&B and what can each role do?

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [28]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
# I do not have access to GPT4, so I will use GPT3.5-turbo instead
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [29]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [30]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [31]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [32]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [33]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [34]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Is it possible to run parallel WANDB runs? I am trying to log an image but it's not appearing online. I am creating
an Image object from RGBA data and then using wandb.log with commit=True. Logging of other images in the same      
project works, just this one does not. What could the problem be? Weights & Biases integration with Gradio works   
within a Jupyter Notebook but not with the same code run as a Python script. Why is this the case?                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
the Team dropdown menu, and select a role type from the Organizational Role dropdown menu.                         

🌆 ]8;id=705467;@site/static/images/app_ui/ezgif-3-b665ff2fa9.gif\ezgif-3-b665ff2fa9.gif]8;;\                                                                                                                    

 5 Select the Add button.                                                                                          

:::info                                                                                                            

 • If you have an Enterprise account, please contact your Account Executive to invite new members to your team. :::


                                               Create a Team Profile                                               

You can customize your team's profile page to show an introduction and showcase reports and projects that are      
visible to the public or team members. Present reports, projects, and external links.                              

 • Highlight your best research to visitors by showcasing your best public reports                                 
 • Showcase the most active projects to make it easier for teammates to find them                                  
 • Find collaborators by adding external links to your company or research lab's website and any papers you've     
   published                                                                                                       




                                                Remove team members                                                

Team admins can open the team settings page and click the delete button next to the departing member's name. Any   
runs that they logged to the team will remain after a user is removed.                                             


                                            Team Roles and Permissions                                             

Select one a team role when you invite colleagues to join a team. There are four team role options:                

 • Admin: Team admins can add and remove other admins or team members. They have permissions to modify all projects
   and full deletion permissions. This includes, but is not limited to, deleting runs, projects, artifacts, and    
   sweeps.                                                                                                         
 • Member: A regular member of the team. A team member is invited by email by the team admin. A team member cannot 
   invite other members. Team members can only delete runs and sweep runs created by that member. Suppose you have 
   two members A and B. Member B moves a Run from team B's project to a different project owned by Member A. Member
   A can not delete the Run Member B moved to Member A'

In [35]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [36]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [37]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

('A user is part of a Weights & Biases team and wants to know which permissions they have to add models to the team registry. They are unsure if they have the necessary permissions, and if not, what they can do to obtain them.\n',
 'How can I add models to the registry of my team in Weights & Biases? Do I need any special permission?\n',
 'To add models to the registry of your team in Weights & Biases, you need to have the "Team Member" or "Team Admin" permission. If you don\'t have this permission, you should contact an existing team admin to request it. Once you have the necessary permission, you can easily add models to the registry and share them with your team.')

In [38]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

In [39]:
wandb.finish()

usage/completion_tokens,▁▂▁▄▅▆█▅▆▆▄▅▆▆▆
usage/elapsed_time,▁▃▁▅▆▆█▆▆▆▆▅▆▆▇
usage/prompt_tokens,▁▂▅▇▇▇█▇▇▇▇▆▇▇▇
usage/total_tokens,▁▂▂▅▆▆█▆▆▇▅▆▆▆▆
usage/completion_tokens,1144
usage/elapsed_time,11.49716
usage/prompt_tokens,915
usage/total_tokens,2059
